# Libraries

In [1]:
library(pROC)
library("ImbTreeAUC")
options(warn=-1)

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var


Loading required package: data.tree

Warning message:
“package ‘data.tree’ was built under R version 4.1.2”
Loading required package: data.table

Loading required package: snow

Loading required package: GenSA

Loading required package: BBmisc

Warning message:
“package ‘BBmisc’ was built under R version 4.1.2”

Attaching package: ‘BBmisc’


The following object is masked from ‘package:base’:

    isFALSE


Loading required package: caret

Warning message:
“package ‘caret’ was built under R version 4.1.2”
Loading required package: ggplot2

Warning message:
“package ‘ggplot2’ was built under R version 4.1.2”
Loading required package: lattice

Loading required package: shiny

Loading required package: shinyjs


Attaching package: ‘shinyjs’


The following object is masked from ‘package:shiny’:

    runExample


The following objects are mask

# Data

In [2]:
Train1 = read.csv("/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Data/x_Miss_train_0.csv",stringsAsFactors=TRUE)
Test1  = read.csv("/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Data/x_Miss_test_0.csv" ,stringsAsFactors=TRUE)

Train2 = read.csv("/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Data/x_Miss_train_1.csv",stringsAsFactors=TRUE)
Test2  = read.csv("/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Data/x_Miss_test_1.csv" ,stringsAsFactors=TRUE)


target = "Dif_Participation"
Train1[,target] = as.factor(Train1[,target])
Train2[,target] = as.factor(Train2[,target])
Test1[,target] = as.factor(Test1[,target])
Test2[,target] = as.factor(Test2[,target])


prob_model1 = read.csv("/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Resultados/Probabilidades_Modelo1.csv")
prob_model2 = read.csv("/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Resultados/Probabilidades_Modelo2.csv")

# Cross Validation

In [11]:
val21 = c()
val22 = c()

for (i in 1:18){
  val_CV = c()
  n = dim(Train1)[1]
  for (j in 1:3){
      
      index0 = 0
      index1 = round((j-1)*n/3)
      index2 = round((j)*n/3)
      index3 = n
  
      train  = Train1[c(index0:index1,index2:index3),]
      x_test = Train1[c(index1:index2),]
      y_test = Train1[c(index1:index2),"Dif_Participation"]
      
      Tree1 <- ImbTreeAUC(Y_name = "Dif_Participation", 
                  X_names =colnames(Train1)[-ncol(Train1)], 
                  data = train, depth = i, min_obs = 5, 
                  type = "AUCg", levelPositive = "1", cp = 0, 
                  n_cores = 1, weights = NULL, cost = NULL, 
                  AUCweight = "none", class_th = "equal", 
                  overfit = "none")
  
      Tree1_pred <- PredictTree(Tree1, x_test)
      val_CV = c(val_CV,auc(y_test,Tree1_pred$"1"))
  }
  
  val21 = c(val21,mean(val_CV))
  n = dim(Train2)[1]
  val_CV2 = c()
  
  for (j in 1:3){
    
    index0 = 0
    index1 = round((j-1)*n/3)
    index2 = round((j)*n/3)
    index3 = n
    
    train  = Train2[c(index0:index1,index2:index3),]
    x_test = Train2[c(index1:index2),]
    y_test = Train2[c(index1:index2),"Dif_Participation"]
    
    Tree2 <- ImbTreeAUC(Y_name = "Dif_Participation", 
                        X_names =colnames(Train2)[-ncol(Train2)], 
                        data = train, depth = i, min_obs = 5, 
                        type = "AUCg", levelPositive = "1", cp = 0, 
                        n_cores = 1, weights = NULL, cost = NULL, 
                        AUCweight = "none", class_th = "equal", 
                        overfit = "none")
    
    Tree2_pred <- PredictTree(Tree2, x_test)
    val_CV2 = c(val_CV2,auc(y_test,Tree2_pred$"1"))
    
  }
  
  val22 = c(val22,mean(val_CV2))
}

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



# Ajuste final

## Modelo 1

In [3]:
Tree1 <- ImbTreeAUC(Y_name = target, 
                    X_names =colnames(Train1)[-ncol(Train1)], 
                    data = Train1, depth = 5, min_obs = 5, 
                    type = "AUCg", levelPositive = "1", cp = 0,
                    n_cores = 1, weights = NULL, cost = NULL, 
                    AUCweight = "none", class_th = "equal", overfit = "none")

Tree1_pred <- PredictTree(Tree1, Test1[,-33])
new_variable = paste("Imbalanced Tree")
prob_model1[new_variable] = Tree1_pred$"1"

## Modelo 2

In [4]:
Tree2 <- ImbTreeAUC(Y_name = target, 
                    X_names =colnames(Train2)[-ncol(Train2)], 
                    data = Train2, depth = 3, min_obs = 5, 
                    type = "AUCg", levelPositive = "1", cp = 0,
                    n_cores = 1, weights = NULL, cost = NULL, 
                    AUCweight = "none", class_th = "equal", overfit = "none")

Tree2_pred <- PredictTree(Tree2, Test2[,-33])
new_variable = paste("Imbalanced Tree")
prob_model2[new_variable] = Tree2_pred$"1"

# Visualización de los árboles

In [4]:
PrintTree(Tree1)
PrintTree(Tree2)

                                                   levelName Number Count Class
1  Root                                                           1  2275     0
2   ¦--Credits = (Si,)                                            2    60     0
3   ¦   ¦--Shy <= 1                                               4    12     0
4   ¦   °--Shy >  1                                               5    48     0
5   ¦       ¦--Shy <= 2                                          10    15     0
6   ¦       ¦   ¦--Year <= 1                                     20     7     0
7   ¦       ¦   °--Year >  1                                     21     8     1
8   ¦       °--Shy >  2                                          11    33     0
9   ¦           ¦--Age <= 22                                     22    25     0
10  ¦           ¦   ¦--Deportiva <= 0                            44    20     0
11  ¦           ¦   °--Deportiva >  0                            45     5     0
12  ¦           °--Age >  22            

# Guardado de datos

In [5]:
write.csv(cbind(prob_model1[,c(2:5,11,6:9,10)]),"/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Resultados/ProbComp_Modelo1.csv", row.names = FALSE)
write.csv(cbind(prob_model2[,c(2:5,11,6:9,10)]),"/Users/miguelaguirrejuarez/Desktop/LUCE/Proyecto/entorno1/Notebooks/TFM/Final/Resultados/ProbComp_Modelo2.csv", row.names = FALSE)